In [3]:
import torch
topic=45
batch_size=1000

dev = "cuda:0"
device = torch.device(dev)

data_atac=torch.load('/data/nelkazwi/lymphoma_data/atac_lymphoma.txt')
theta=torch.load("/scratch/nelkazwi/lymphoma_data/"+str(topic)+"_topics/theta_"+str(topic)+".txt")
phi=torch.load("/scratch/nelkazwi/lymphoma_data/"+str(topic)+"_topics/phi_"+str(topic)+".txt")
theta=theta[50:,:,:]
phi=phi[50:,:,:]


n_samples=phi.shape[0]
n=theta.shape[2]
P=data_atac.shape[1]
L=torch.zeros(P,dtype=torch.double,device=device)
V=torch.zeros(P,device=device)
V_2=torch.zeros(P,device=device)
n_cells=theta.shape[1]    

n_batches=n_cells+batch_size-(n_cells%batch_size)
c=torch.arange(0,n_batches,batch_size)
c=torch.hstack((c,torch.tensor(n_cells)))
c=c.to(device)

a,b,rep_c=torch.unique(data_atac[0,:],return_inverse =True,return_counts=True)
rep_c=rep_c.type(torch.LongTensor) 
ren=torch.arange(n_cells)
rep_c_=torch.repeat_interleave(ren, rep_c,dim=0)


k_atac=torch.zeros(c.shape[0],dtype=torch.int64)

q=1
t=0
t_=0
for i in  torch.arange(batch_size,n_batches,batch_size):
    
    k_atac[q]=t+torch.sum(rep_c[t_:i])
    t=int(k_atac[q].item())
    t_=i
    q+=1
k_atac[q]=t+torch.sum(rep_c[t_:])

indices=torch.zeros([data_atac.shape[1]])
indices2=torch.zeros([data_atac.shape[1]])
regions=torch.tensor([])
region_rep=torch.tensor([])
region_rep_=torch.tensor([])
region_batching=torch.zeros([k_atac.shape[0]])
for i in torch.arange(1,k_atac.shape[0]):
    
   sorted, indices[int(k_atac[i-1].item()):int(k_atac[i].item())]= torch.sort(
       data_atac[1,int(k_atac[i-1].item()):int(k_atac[i].item())])
   sorted2,indices2[int(k_atac[i-1].item()):int(k_atac[i].item())]=torch.sort(
       indices[int(k_atac[i-1].item()):int(k_atac[i].item())])
   a,b,rep_r=torch.unique(
       sorted,return_inverse =True,return_counts=True)
   regions=torch.cat((regions,a),0)
   region_batching[i]=a.shape[0]+region_batching[i-1] ##list of regiosn per batch
   region_rep=torch.cat((region_rep,rep_r),0) ##how many times presented region in a batch are repeated
   rep_r=rep_r.type(torch.LongTensor) 
   ren=torch.arange(a.shape[0]) 
   rep_r_=torch.repeat_interleave(ren, rep_r,dim=0)
   region_rep_=torch.cat((region_rep_,rep_r_),0) 

region_rep=region_rep.type(torch.LongTensor) 
regions=regions.type(torch.LongTensor) 
indices=indices.int()
indices2=indices2.int()
region_rep_=region_rep_.type(torch.int64)


region_rep=region_rep.to(device)
regions=regions.to(device)
indices=indices.to(device)
indices2=indices2.to(device)
rep_c=rep_c.to(device)
rep_c_=rep_c_.to(device)
region_rep_=region_rep_.to(device)


for sample in range(0,n_samples):
    theta_tmp=theta[sample,:,:].to(device)
    phi_tmp=phi[sample,:,:].to(device)
    for i in torch.arange(1,c.shape[0]):
        theta_=theta_tmp[c[i-1]:c[i],:]
        phi_=phi_tmp[:,regions[int(region_batching[i-1].item()):int(region_batching[i].item())]]
        n_regions=phi_.shape[1]
        z_atac_=torch.repeat_interleave(
        theta_, rep_c[c[i-1]:c[i]],dim=0)##cells in this batch
        phi_=torch.repeat_interleave(
           phi_,region_rep[int(region_batching[i-1].item()):int(region_batching[i].item())],dim=1)##regions in this batch
        phi_=torch.index_select(phi_,1, indices2[int(k_atac[i-1].item()):int(k_atac[i].item())])##reorder the regions acording to data
        z_atac_=torch.multiply(z_atac_.T,phi_)
        
        z_atac_=torch.sum(z_atac_,axis=0)
        L[int(k_atac[i-1].item()):int(k_atac[i].item())]+=z_atac_
        V[int(k_atac[i-1].item()):int(k_atac[i].item())]+=torch.pow(torch.log(z_atac_),2)
        V_2[int(k_atac[i-1].item()):int(k_atac[i].item())]+=torch.log(z_atac_)
WAIC=-2*(torch.sum(torch.log(L/n_samples))-(torch.sum(V)-torch.sum(torch.pow(V_2,2)/n_samples))/(n_samples-1))
torch.save(WAIC,"/scratch/nelkazwi/lymphoma_data/"+str(n)+"_topics/WAIC_r_"+str(n)+".txt")